In [1]:
from transformers import pipeline
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


image_classifier = pipeline('image-classification', device = device,
                            model="MattyB95/VIT-VoxCelebSpoof-Mel_Spectrogram-Synthetic-Voice-Detection",
                            feature_extractor="MattyB95/VIT-VoxCelebSpoof-Mel_Spectrogram-Synthetic-Voice-Detection")

In [2]:
from conf.configure import Config
import pandas as pd
import os
from torch.utils.data import Dataset, DataLoader

CONFIG = Config()

test_df = pd.read_csv(os.path.join(CONFIG.ROOT_FOLDER, 'test.csv'))

from dataset.spectrogram_dataset import AudioDataset

test_dataset = AudioDataset(test_df, False, CONFIG)

#test_loader = DataLoader(test_dataset, shuffle=False, batch_size=CONFIG.BATCH_SIZE, num_workers = 4)

In [3]:
from tqdm import tqdm
from torch.nn.functional import softmax

out = []

for i, item in tqdm(enumerate(test_dataset)) :
    pred = image_classifier(item)
    spoof_score = pred[0]["score"]
    bonafide_score = pred[1]["score"]
    out.append([spoof_score, bonafide_score])
submit = pd.read_csv(os.path.join(CONFIG.ROOT_FOLDER,'sample_submission.csv'))
submit.iloc[:, 1:] = out
print(submit.head())
submit.to_csv('./submit.csv', index=False)


0it [00:00, ?it/s]/tmp/ipykernel_98556/4279195804.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = softmax(torch.tensor([spoof_score, bonafide_score]))
3it [00:01,  2.06it/s]

3.839230373614555e-07
tensor([0.7311, 0.2689])
3.885160708705371e-07
tensor([0.7311, 0.2689])
3.9235678173099586e-07
tensor([0.7311, 0.2689])


6it [00:02,  4.75it/s]

3.845775040645094e-07
tensor([0.7311, 0.2689])
3.959567322908697e-07
tensor([0.7311, 0.2689])
3.888299886511959e-07
tensor([0.7311, 0.2689])
3.824459611223574e-07
tensor([0.7311, 0.2689])


8it [00:02,  3.70it/s]


3.7124607388250297e-07
tensor([0.7311, 0.2689])


KeyboardInterrupt: 